In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENWEATHERMAP_API_KEY"] = os.getenv("OPENWEATHERMAP_API_KEY")
os.environ["ALPHAVANTAGE_API_KEY"] = os.getenv("ALPHAVANTAGE_API_KEY")

In [ ]:
from langchain.tools import tool
# from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState,StateGraph, END, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from IPython.display import Image, display
import yfinance as yf
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re
from langchain_groq import ChatGroq
import pyowm
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper


In [ ]:
from pyowm.owm import OWM

# Replace with your API key
owm = OWM(os.getenv("OPENWEATHERMAP_API_KEY"))
mgr = owm.weather_manager()

place = "Dubai"
# Search for weather in London
mgr = owm.weather_manager()
observation = mgr.weather_at_place(place)
w = observation.weather

# Get temperature in Celsius
temp = w.temperature('celsius')
print(temp)

# Get weather status
print(w.detailed_status)



In [ ]:
import os

from langchain_community.utilities import OpenWeatherMapAPIWrapper

weather = OpenWeatherMapAPIWrapper()

weather_data = weather.run("Kolhapur")
print(weather_data)





In [ ]:

# Replace 'YOUR_API_KEY' with your actual API key
owm = pyowm.OWM(os.getenv("OPENWEATHERMAP_API_KEY"))
mgr = owm.weather_manager()

# Get weather forecast for a location (e.g., London, UK)
observation = mgr.weather_at_place('London,UK')
weather = observation.weather

# Print weather details
print(f"Location: {observation.location.name}")
print(f"Temperature: {weather.temperature('celsius')['temp']}°C")
print(f"Status: {weather.status}")
print(f"Humidity: {weather.humidity}%")
print(f"Wind Speed: {weather.wind()['speed']} m/s")

In [5]:

llm = ChatGroq(
    model_name="deepseek-r1-distill-llama-70b",
    temperature=0
)

In [ ]:
class User:
    def __init__(self, destination, trip_start_date, trip_end_date, number_of_travelers, native_currency, maximum_budget):
        self.destination = destination
        self.trip_start_date = trip_start_date
        self.trip_end_date = trip_end_date
        self.number_of_travelers = number_of_travelers
        self.native_currency = native_currency
        self.maximum_budget = maximum_budget
        
    def get_user_travel_details(self):
        return f"User want to travel to {self.destination} for duration {self.trip_start_date} to {self.trip_end_date} with {self.number_of_travelers} people. User's native country currency is {self.native_currency} and user has budget constrain of {self.maximum_budget}"

In [ ]:
user1 = User("London","10 July 2025", "17 July 2025", 3, "Indian Rupees", "1000 Euros")
user1.get_user_travel_details()

In [7]:
class AttractionActivityManager:
    
    # @tool
    def search_attraction_and_activities(self, destination):
        """
        Search for attractions, activities, dining options, and transportation details for a specified travel destination .

        This function performs the following:
        - Retrieves a list of popular tourist attractions in the travel destination .
        - Recommends the best restaurants in the travel destination .
        - Suggests activities or things to do in the travel destination .
        - Offers details about the transportation options available within the travel destination  (such as buses, taxis, metro, etc.).

        Args:
            destination (str): Name of the destination the user plans to travel to.

        Returns:
            attractions, activities, dining options, and transportation details for a city in multiline string format
        """
        
        parser = StrOutputParser()
        template = """
        You are a travel expert. Provide helpful, accurate, realistic, and up-to-date information for tourists.

        Your response should follow this format:

        -------------------------------------------------------------------------------------------
        Top Attractions and Activities

        Attractions

        [Attraction Name]
        Description: [Brief description]
        Cost: [Cost per person in USD]
        Hours: [Hours of operation]

        [Repeat for minimum 10 attractions]

        Restaurants

        [Restaurant Name]
        Cuisine: [Type of cuisine]
        Cost: [Cost per person in USD]
        Hours: [Hours of operation]

        [Repeat for minimum 10 restaurants]

        Activities

        [Activity Name]
        Description: [Brief description]
        Cost: [Cost per person in USD]
        Duration: [Duration]

        [Repeat for minimum 10 activities]

        Transportation

        [Option Name]
        Description: [Brief description]
        Cost: [Cost per person or per ride in USD]
        Recommendation: [Additional helpful information]

        -------------------------------------------------------------------------------------------
        Consider the following context:
        - Desired destination: {destination}

        Fill in the details with realistic, up-to-date information for the specified destination.

        Make sure your response sticks closely to the format described above.
        
        All the cost should be given in USD

        """
        
        prompt = PromptTemplate(
            template=template,
            input_variable=[ "destination"]
        )
        
        chain = prompt | llm | parser
        response = chain.invoke({"destination":destination})
        
        # Remove <think>...</think> tags and their content from LLM response.
        return re.sub(r'<think>.*?</think>', '', response, flags=re.MULTILINE | re.DOTALL)


In [8]:
obj = AttractionActivityManager()
obj.search_attraction_and_activities("Mumbai")

"\n\nCertainly! Here's a well-organized and elegant presentation of the travel information for Mumbai, structured according to the user's format:\n\n---\n\n**Mumbai Travel Guide**\n\n**Top Attractions and Activities**\n\n**Attractions**\n\n1. **Gateway of India**\n   - Description: Iconic landmark overlooking the Arabian Sea.\n   - Cost: Free\n   - Hours: 24/7\n\n2. **Haji Ali Dargah**\n   - Description: Beautiful mosque located in the sea, accessible by a causeway.\n   - Cost: Free\n   - Hours: 5:30 AM - 10:00 PM\n\n3. **Chhatrapati Shivaji Maharaj Vastu Sangrahalaya (CSMVS)**\n   - Description: Museum showcasing Indian history and art.\n   - Cost: $2 - $3\n   - Hours: 10:15 AM - 6:00 PM\n\n4. **Marine Drive**\n   - Description: Scenic promenade along the Arabian Sea.\n   - Cost: Free\n   - Hours: 24/7\n\n5. **Colaba Causeway**\n   - Description: Bustling shopping street with diverse shops.\n   - Cost: Free\n   - Hours: 10:00 AM - 10:00 PM\n\n6. **Siddhivinayak Temple**\n   - Descript

In [ ]:
class WeatherService:
    
    def __init__(self, api_key=None):
        self.api_key = api_key or os.getenv("OPENWEATHERMAP_API_KEY")
        self.owm = pyowm.OWM(self.api_key)
        self.mgr = self.owm.weather_manager()

    def _get_manager(self):
        """Helper to reuse the Weather Manager instance."""
        return self.mgr

    @tool
    def get_current_weather(self, city: str) -> str:
        """
        Retrieve the current weather conditions for a specified city.

        This method connects to the OpenWeatherMap API using the provided API key
        and fetches the current temperature, atmospheric conditions, humidity, and 
        wind speed for the specified city.

        Args:
            city (str): Name of the city (and country code, if applicable) for which 
                the current weather is to be retrieved. 
                For example: "London,UK" or "Sydney,AU".

        Returns:
            str: A multi-line string with the following details:
                - Location (City)
                - Temperature in Celsius
                - Weather Status (Clear, Rain, Cloudy, etc.)
                - Humidity in percent
                - Wind Speed in m/s
        """
        
        try:
            observation = self.mgr.weather_at_place(city)
            weather = observation.weather

            return f"""Location: {observation.location.name}
            Temperature: {weather.temperature('celsius')['temp']}°C
            Status: {weather.status}
            Humidity: {weather.humidity}%
            Wind Speed: {weather.wind()['speed']} m/s
            """
        except Exception as e:
            return f"Error retrieving current weather for {city}. Exception: {e}"

    
    @tool
    def get_weather_forecast(self, city: str) -> str:
        """
        Retrieve the 5-day weather forecast in 3-hour intervals for a specified city.

        This method connects to the OpenWeatherMap API using the provided API key
        and fetches the 5-day forecast with data points at 3-hour intervals. 
        Each entry includes the timestamp, temperature in Celsius, and 
        the atmospheric condition (status).

        Args:
            city (str): Name of the city (and country code, if applicable) for which 
                the 5-day forecast is to be retrieved. 
                For example: "London,UK" or "Sydney,AU".

        Returns:
            str: A multi-line string with the following details for each 3-hour interval:
                - Time (UTC)
                - Temperature in Celsius
                - Weather Status
                - Separator ("---") between intervals
        """
        try:
            forecast = self.mgr.forecast_at_place(city, interval='3h')
            forecast_report = ""

            for weather_data in forecast.forecast:
                forecast_report += f"Time: {weather_data.reference_time('iso')}\n"
                forecast_report += f"Temperature: {weather_data.temperature('celsius')['temp']}°C\n"
                forecast_report += f"Status: {weather_data.status}\n"
                forecast_report += "---\n"

            return forecast_report
        except Exception as e:
            return f"Error retrieving forecast for {city}. Exception: {e}"



In [20]:
obj = WeatherService(api_key=os.getenv("OPENWEATHERMAP_API_KEY"))
response1 = obj.get_current_weather("Mumbai")
print(response1)

Location: Mumbai
            Temperature: 29.05°C
            Status: Clouds
            Humidity: 79%
            Wind Speed: 10.31 m/s
            


In [21]:
response2 = obj.get_weather_forecast("Sidney")
print(response2)

Time: 2025-06-17 12:00:00+00:00
Temperature: 11.61°C
Status: Clouds
---
Time: 2025-06-17 15:00:00+00:00
Temperature: 9.55°C
Status: Clouds
---
Time: 2025-06-17 18:00:00+00:00
Temperature: 6.38°C
Status: Clear
---
Time: 2025-06-17 21:00:00+00:00
Temperature: 5.12°C
Status: Clear
---
Time: 2025-06-18 00:00:00+00:00
Temperature: 11.45°C
Status: Clear
---
Time: 2025-06-18 03:00:00+00:00
Temperature: 15.23°C
Status: Clear
---
Time: 2025-06-18 06:00:00+00:00
Temperature: 14.22°C
Status: Clear
---
Time: 2025-06-18 09:00:00+00:00
Temperature: 12.33°C
Status: Clear
---
Time: 2025-06-18 12:00:00+00:00
Temperature: 11.34°C
Status: Clear
---
Time: 2025-06-18 15:00:00+00:00
Temperature: 8.97°C
Status: Clear
---
Time: 2025-06-18 18:00:00+00:00
Temperature: 7.99°C
Status: Clear
---
Time: 2025-06-18 21:00:00+00:00
Temperature: 7.44°C
Status: Clouds
---
Time: 2025-06-19 00:00:00+00:00
Temperature: 11.81°C
Status: Clouds
---
Time: 2025-06-19 03:00:00+00:00
Temperature: 15.45°C
Status: Clouds
---
Time: 2

In [ ]:
class BookingScraper:
    def __init__(self):
        # Headers to mimic a browser and enforce US English
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',  # Prioritize US English
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
        }
        self.session = requests.Session()  # Persistent session for cookies

    def _set_currency_cookie(self, currency: str = 'USD'):
        """Set currency cookie for the session to enforce USD."""
        try:
            url = f"https://www.booking.com/?change_currency=1;selected_currency={currency};top_currency=1"
            response = self.session.get(url, headers=self.headers)
            if response.status_code != 200:
                print(f"Failed to set currency cookie. Status code: {response.status_code}")
        except Exception as e:
            print(f"Error setting currency cookie: {str(e)}")

    def find_hotels(self, city: str, checkin_date: str, checkout_date: str, num_adults: int, max_budget: float = None, sort_by: str = None) -> list:
        """
        Find hotels in a given city for specific check-in and check-out dates and number of adults.
        Data is in English, prices in USD, with optional budget and sorting filters.
        Includes room details (room type, bed types, breakfast, cancellation, availability) in description.
        
        Args:
            city (str): Name of the city (e.g., "New York")
            checkin_date (str): Check-in date in YYYY-MM-DD format
            checkout_date (str): Check-out date in YYYY-MM-DD format
            num_adults (int): Number of adults for the booking
            max_budget (float, optional): Maximum price per stay in USD
            sort_by (str, optional): Sorting option ('price_low', 'rating_high', 'price_high')
        
        Returns:
            list: List of dictionaries containing hotel details (name, price, reviews, description)
        """
        # Validate inputs
        try:
            datetime.strptime(checkin_date, '%Y-%m-%d')
            datetime.strptime(checkout_date, '%Y-%m-%d')
        except ValueError:
            raise ValueError("Dates must be in YYYY-MM-DD format")
        
        if not isinstance(num_adults, int) or num_adults < 1:
            raise ValueError("Number of adults must be a positive integer")
        
        if max_budget is not None and (not isinstance(max_budget, (int, float)) or max_budget <= 0):
            raise ValueError("Max budget must be a positive number")

        # Set currency cookie to USD
        self._set_currency_cookie()

        # Map user-friendly sort options to Booking.com's order parameter
        sort_options = {
            'price_low': 'price',  # Lowest price first
            'price_high': 'price_desc',  # Highest price first
            'rating_high': 'review_score'  # Highest rating first
        }
        sort_param = f"&order={sort_options[sort_by]}" if sort_by in sort_options else ""

        # Format the Booking.com search URL
        city = city.replace(' ', '+')
        url = (
            f"https://www.booking.com/searchresults.en-us.html?"  # Set language to English
            f"ss={city}&"
            f"checkin={checkin_date}&"
            f"checkout={checkout_date}&"
            f"group_adults={num_adults}&"
            f"no_rooms=2&"  # Kept 2 rooms as per your code
            f"currency=USD"  # Explicitly set currency to USD
            f"{sort_param}"
        )

        try:
            # Send GET request with session
            response = self.session.get(url, headers=self.headers)
            if response.status_code != 200:
                raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

            # Parse HTML with BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            hotels = []

            # Find hotel cards (limit to 11 as in original code)
            hotel_cards = soup.find_all('div', {'data-testid': 'property-card'})[:11]
            if not hotel_cards:
                return hotels  # Return empty list if no hotels found

            for card in hotel_cards:
                hotel = {}

                # Extract hotel name
                name_tag = card.find('div', {'data-testid': 'title'})
                hotel['name'] = name_tag.text.strip() if name_tag else 'N/A'

                # Extract price
                price_tag = card.find('span', {'data-testid': 'price-and-discounted-price'})
                price_text = price_tag.text.strip() if price_tag else 'N/A'
                hotel['price'] = price_text

                # Extract price value for budget filtering
                if price_text != 'N/A':
                    # Extract numeric value (e.g., "US$1,234" -> 1234.0)
                    price_value = float(re.sub(r'[^\d.]', '', price_text.replace(',', '')))
                else:
                    price_value = float('inf')  # Treat N/A as infinite for filtering

                # Extract rating details
                rating_tag = card.find('div', {'data-testid': 'review-score'})
                if rating_tag:
                    score = rating_tag.select_one("div.bc946a29db")
                    label = rating_tag.select_one("div.f63b14ab7a.f546354b44.becbee2f63")
                    reviews = rating_tag.select_one("div.fff1944c52.fb14de7f14.eaa8455879")

                    score = score.text.strip().replace("Scored ", '') if score else "N/A"
                    label = label.text.strip() if label else "N/A"
                    reviews = reviews.text.strip() if reviews else "N/A"

                    hotel['reviews'] = f"For total {reviews} rating is {score} which is {label}."
                    rating_score = float(score) if score.replace('.', '').isdigit() else 0.0
                else:
                    hotel['reviews'] = "N/A"
                    rating_score = 0.0

                # Extract room details for description
                description_parts = []
                
                # Room type
                room_tag = card.find('h4', {'data-testid': 'recommended-units'})
                room_type = room_tag.text.strip() if room_tag else 'Standard Room'
                description_parts.append(room_type)

                # Bed types
                bed_tag = card.find('div', {'data-testid': 'property-card-unit-configuration'})
                bed_types = bed_tag.text.strip() if bed_tag else 'Multiple bed types'
                description_parts.append(bed_types)

                # Breakfast info
                breakfast_tag = card.find('div', {'data-testid': 'mealplan-block'})
                breakfast = 'Breakfast included' if breakfast_tag and 'breakfast' in breakfast_tag.text.lower() else 'Breakfast not included'
                description_parts.append(breakfast)

                # Cancellation policy
                cancellation_tag = card.find('div', {'data-testid': 'free-cancellation-block'})
                cancellation = 'Free cancellation' if cancellation_tag else 'Non-refunded'
                description_parts.append(cancellation)

                # Availability info
                availability_tag = card.find('div', {'data-testid': 'availability-cta'})
                availability = availability_tag.text.strip() if availability_tag else 'Availability not specified'
                description_parts.append(availability)

                # Combine into description
                hotel['description'] = ', '.join(description_parts)

                # Apply budget filter
                if max_budget is None or price_value <= max_budget:
                    hotel['price_value'] = price_value  # Temporary for sorting
                    hotel['rating_score'] = rating_score  # Temporary for sorting
                    hotels.append(hotel)

            # Sort hotels in-memory if needed
            if sort_by == 'rating_high':
                hotels.sort(key=lambda x: x['rating_score'], reverse=True)
            elif sort_by == 'price_low':
                hotels.sort(key=lambda x: x['price_value'])
            elif sort_by == 'price_high':
                hotels.sort(key=lambda x: x['price_value'], reverse=True)

            # Remove temporary keys (price_value, rating_score)
            hotels = [{k: v for k, v in hotel.items() if k in ['name', 'price', 'reviews', 'description']} for hotel in hotels]

            return hotels

        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return []

def accomodation_calculator():
    scraper = BookingScraper()
    city = "New York"
    checkin = "2025-08-01"
    checkout = "2025-08-03"
    adults = 2
    budget = 2000.0  # Max $2000 USD
    sort_option = "rating_high"  # Sort by highest rating
    
    hotels = scraper.find_hotels(city, checkin, checkout, adults, max_budget=budget, sort_by=sort_option)
    # for hotel in hotels:
    #     print(f"Hotel: {hotel['name']}, \n      Price: {hotel['price']}, \n      Reviews: {hotel['reviews']}, \n      Description: {hotel['description']}")
    return hotels

if __name__ == "__main__":
    accomodation_calculator()

Hotel: Hilton Club New York, 
      Price: US$542, 
      Reviews: For total 385 reviews rating is 8.7 which is Excellent., 
      Description: Standard Room, Multiple bed types, Breakfast not included, Non-refunded, See availability
Hotel: The Mayfair Hotel Times Square, Ascend Hotel Collection, 
      Price: US$370, 
      Reviews: For total 2,520 reviews rating is 8.5 which is Very Good., 
      Description: Standard Room, Multiple bed types, Breakfast not included, Non-refunded, See availability
Hotel: Warwick New York, 
      Price: US$557, 
      Reviews: For total 6,901 reviews rating is 8.3 which is Very Good., 
      Description: Standard Room, Multiple bed types, Breakfast not included, Non-refunded, See availability
Hotel: voco Times Square South New York by IHG, 
      Price: US$404, 
      Reviews: For total 3,845 reviews rating is 8.2 which is Very Good., 
      Description: Standard Room, Multiple bed types, Breakfast not included, Non-refunded, See availability
Hotel: T

In [ ]:

class ExpenseCalculator:
    @tool
    def add(self):
        """Add expenses"""
        pass
    
    @tool
    def multiply(self):
        """Multiply costs (e.g., daily x days)"""
        pass
    
    @tool
    def calculate_total_cost(self):
        """Calculate total trip cost"""
        pass
    
    @tool
    def calculate_daily_budget(self):
        """Calculate daily spending budget"""
        pass



In [ ]:

class CurrencyService:
    
    def convert_currency(self, user_currency, destination_currency, amount):
        alpha_vantage = AlphaVantageAPIWrapper()
        # INR to USD conversion
        rate_dictionary = alpha_vantage._get_exchange_rate(user_currency, destination_currency)
        rate = float(rate_dictionary['Realtime Currency Exchange Rate']['5. Exchange Rate'])
        print(rate)
        converted_amount = amount*rate
        print(f"for {amount} {user_currency} with exchange rate {rate} converted amount is {converted_amount}")
        
# obj = CurrencyService()
# obj.convert_currency("INR", "USD", 8600)
# obj.convert_currency("INR","JPY",1000)

0.0116
for 8600 INR with exchange rate 0.0116 converted amount is 99.75999999999999
1.6743
for 1000 INR with exchange rate 1.6743 converted amount is 1674.3


In [ ]:

class ItineraryPlanner:
    @tool
    def get_day_plan(self):
        """Generate daily itinerary"""
        pass
    
    @tool
    def create_full_itinerary(self):
        """Create complete multi-day itinerary"""
        pass



In [ ]:

class TripSummarizer:
    @tool
    def create_trip_summary(self):
        """Generate comprehensive trip summary"""
        pass



In [ ]:

class TravelPlanCompiler:
    @tool
    def return_complete_travel_plan(self):
        """Compile and return full travel plan"""
        pass

In [ ]:
# Get the 5-day forecast with 3-hour intervals
forecast = mgr.forecast_at_place('London,UK', interval='3h')

# Iterate through the forecast
for weather_data in forecast.forecast:
    print(f"Time: {weather_data.reference_time('iso')}")
    print(f"Temperature: {weather_data.temperature('celsius')['temp']}°C")
    print(f"Status: {weather_data.status}")
    print("---")

In [ ]:
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from langchain.agents import load_tools
# tools = load_tools(["openweathermap-api"])

In [ ]:

class BookingScraper:
    def __init__(self):
        # Headers to mimic a browser and enforce US English
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',  # Prioritize US English
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
        }
        self.session = requests.Session()  # Persistent session for cookies

    def _set_currency_cookie(self, currency: str = 'USD'):
        """Set currency cookie for the session to enforce USD."""
        try:
            url = f"https://www.booking.com/?change_currency=1;selected_currency={currency};top_currency=1"
            response = self.session.get(url, headers=self.headers)
            if response.status_code != 200:
                print(f"Failed to set currency cookie. Status code: {response.status_code}")
        except Exception as e:
            print(f"Error setting currency cookie: {str(e)}")

    def find_hotels(self, city: str, checkin_date: str, checkout_date: str, num_adults: int, max_budget: float = None, sort_by: str = None) -> list:
        """
        Find hotels in a given city for specific check-in and check-out dates and number of adults.
        Data is in English, prices in USD, with optional budget and sorting filters.
        
        Args:
            city (str): Name of the city (e.g., "Goa")
            checkin_date (str): Check-in date in YYYY-MM-DD format
            checkout_date (str): Check-out date in YYYY-MM-DD format
            num_adults (int): Number of adults for the booking
            max_budget (float, optional): Maximum price per stay in USD
            sort_by (str, optional): Sorting option ('price_low', 'rating_high', 'price_high')
        
        Returns:
            list: List of dictionaries containing hotel details
        """
        # Validate inputs
        try:
            datetime.strptime(checkin_date, '%Y-%m-%d')
            datetime.strptime(checkout_date, '%Y-%m-%d')
        except ValueError:
            raise ValueError("Dates must be in YYYY-MM-DD format")
        
        if not isinstance(num_adults, int) or num_adults < 1:
            raise ValueError("Number of adults must be a positive integer")
        
        if max_budget is not None and (not isinstance(max_budget, (int, float)) or max_budget <= 0):
            raise ValueError("Max budget must be a positive number")

        # Set currency cookie to USD
        self._set_currency_cookie()

        # Map user-friendly sort options to Booking.com's order parameter
        sort_options = {
            'price_low': 'price',  # Lowest price first
            'price_high': 'price_desc',  # Highest price first
            'rating_high': 'review_score'  # Highest rating first
        }
        sort_param = f"&order={sort_options[sort_by]}" if sort_by in sort_options else ""

        # Format the Booking.com search URL
        city = city.replace(' ', '+')
        url = (
            f"https://www.booking.com/searchresults.en-us.html?"  # Set language to English
            f"ss={city}&"
            f"checkin={checkin_date}&"
            f"checkout={checkout_date}&"
            f"group_adults={num_adults}&"
            f"no_rooms=1&"
            f"currency=USD"  # Explicitly set currency to USD
            f"{sort_param}"
        )

        try:
            # Send GET request with session
            response = self.session.get(url, headers=self.headers)
            if response.status_code != 200:
                raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

            # Parse HTML with BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            hotels = []

            # Find hotel cards (limit to 11 as in original code)
            hotel_cards = soup.find_all('div', {'data-testid': 'property-card'})[:11]
            if not hotel_cards:
                return hotels  # Return empty list if no hotels found

            for card in hotel_cards:
                hotel = {}

                # Extract hotel name
                name_tag = card.find('div', {'data-testid': 'title'})
                hotel['name'] = name_tag.text.strip() if name_tag else 'N/A'

                # Extract price
                price_tag = card.find('span', {'data-testid': 'price-and-discounted-price'})
                price_text = price_tag.text.strip() if price_tag else 'N/A'
                hotel['price'] = price_text

                # Extract price value for budget filtering
                if price_text != 'N/A':
                    # Extract numeric value (e.g., "$1,234" -> 1234.0)
                    price_value = float(re.sub(r'[^\d.]', '', price_text.replace(',', '')))
                    hotel['price_value'] = price_value
                else:
                    hotel['price_value'] = float('inf')  # Treat N/A as infinite for filtering

                # Extract rating details
                rating_tag = card.find('div', {'data-testid': 'review-score'})
                if rating_tag:
                    score = rating_tag.select_one("div.bc946a29db")
                    label = rating_tag.select_one("div.f63b14ab7a.f546354b44.becbee2f63")
                    reviews = rating_tag.select_one("div.fff1944c52.fb14de7f14.eaa8455879")

                    score = score.text.strip().replace("Scored ", '') if score else "N/A"
                    label = label.text.strip() if label else "N/A"
                    reviews = reviews.text.strip() if reviews else "N/A"

                    hotel['description'] = f"For total {reviews} rating is {score} which is {label}."
                    hotel['rating_score'] = float(score) if score.replace('.', '').isdigit() else 0.0
                else:
                    hotel['description'] = "N/A"
                    hotel['rating_score'] = 0.0

                # Apply budget filter
                if max_budget is None or hotel['price_value'] <= max_budget:
                    hotels.append(hotel)

            # Sort hotels in-memory if needed
            if sort_by == 'rating_high':
                hotels.sort(key=lambda x: x['rating_score'], reverse=True)
            elif sort_by == 'price_low':
                hotels.sort(key=lambda x: x['price_value'])
            elif sort_by == 'price_high':
                hotels.sort(key=lambda x: x['price_value'], reverse=True)

            return hotels

        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return []

# Example usage
if __name__ == "__main__":
    scraper = BookingScraper()
    city = "Goa"
    checkin = "2025-08-01"
    checkout = "2025-08-03"
    adults = 2
    budget = 200.0  # Max $200 USD
    sort_option = "rating_high"  # Sort by lowest price
    
    hotels = scraper.find_hotels(city, checkin, checkout, adults, max_budget=budget, sort_by=sort_option)
    for hotel in hotels:
        print(f"Hotel: {hotel['name']}, \n      Price: {hotel['price']}, \n      Rating: {hotel['description']}")

Hotel: Maashpi Hotel & Resort, 
      Price: US$74, 
      Rating: For total 5 reviews rating is 10 which is Exceptional.
Hotel: La Utopia Resort, 
      Price: US$32, 
      Rating: For total 5 reviews rating is 10 which is Exceptional.
Hotel: Etereo Stays, Luxury Premium Apartments Baga, Arpora, Goa, 
      Price: US$93, 
      Rating: For total 6 reviews rating is 10 which is Exceptional.
Hotel: Richita resort, 
      Price: US$46, 
      Rating: For total 6 reviews rating is 10 which is Exceptional.
Hotel: Alaya Stays 1 BHK with Balcony & Pool - Veera Marina, 
      Price: US$133, 
      Rating: For total 10 reviews rating is 10 which is Exceptional.
Hotel: Mela By Half, 
      Price: US$69, 
      Rating: For total 9 reviews rating is 10 which is Exceptional.
Hotel: Elements by Rosetta, 
      Price: US$180, 
      Rating: For total 5 reviews rating is 10 which is Exceptional.
Hotel: Kay's Calangute Resort Service Apartments, 
      Price: US$55, 
      Rating: For total 5 reviews

In [ ]:
https://www.easemytrip.com/hotel-new/search?e=2025617165955&city=Pune,%20India&cin=18%2F06%2F2025&cOut=19%2F06%2F2025&Hotel=NA&Rooms=1&pax=2